# Basic Prompt Structures Tutorial

## Overview

This tutorial explores two essential prompt structures used in AI interactions:
1. Single-turn prompts
2. Multi-turn prompts (conversations)

We'll utilize Amazon Nova via OpenRouter and LangChain to illustrate these concepts.

## Motivation

Grasping various prompt structures is essential for effective AI communication. Single-turn prompts excel in quick, direct queries, while multi-turn prompts facilitate more nuanced, context-rich exchanges. Proficiency in these structures enhances the versatility and efficacy of AI applications across diverse scenarios.

## Key Components

1. **Single-turn Prompts**: One-time interactions with the AI model.
2. **Multi-turn Prompts**: Sequential exchanges that preserve context.
3. **Prompt Templates**: Standardized structures for consistent AI querying.
4. **Conversation Chains**: Techniques for maintaining context across multiple interactions.

## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
from os import getenv
from typing import List

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI

load_dotenv()


# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="bedrock/nova-lite-v1",
)

## 1. Single-turn Prompts

Single-turn prompts are one-shot interactions with the language model. They consist of a single input (prompt) and generate a single output (response).

In [3]:
single_turn_prompt = "What are the three primary colors?"
print(llm.invoke(single_turn_prompt).content)

The three primary colors are red, blue, and yellow. These colors are fundamental in both additive and subtractive color mixing systems.

1. **Additive Color Mixing**:
   - Used in light-based systems (e.g., computer screens, projectors).
   - When combined, red, blue, and yellow light create white light.

2. **Subtractive Color Mixing**:
   - Used in pigment-based systems (e.g., painting, printing).
   - When combined, red, blue, and yellow pigments create black or a dark color.


Now, let's use a PromptTemplate to create a more structured single-turn prompt:

In [4]:
structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Provide a brief explanation of {topic} and list its three main components.",
)

chain = structured_prompt | llm
print(chain.invoke({"topic": "color theory"}).content)

Color theory is a framework that explains how colors interact and the effects they produce when combined. It is essential in various fields such as art, design, and even marketing. Understanding color theory allows individuals to create harmonious and visually appealing compositions.

The three main components of color theory are:

1. **The Color Wheel**: This is a circular diagram that represents the spectrum of colors. It is divided into primary, secondary, and tertiary colors. The primary colors (red, blue, and yellow) cannot be created by mixing other colors and serve as the foundation for all other colors. Secondary colors (green, orange, and purple) are formed by mixing two primary colors, while tertiary colors result from mixing a primary color with a secondary color.

2. **Color Harmony**: This refers to the pleasing arrangement of colors. It involves understanding how different colors interact to create balance and aesthetic appeal. Common types of color harmony include comple

You can learn more about LangChain Expression Language from [LCEL](https://python.langchain.com/docs/concepts/lcel/)

## 2. Multi-turn Prompts (Conversations)

Multi-turn prompts involve a series of interactions with the language model, allowing for more complex and context-aware conversations.

In [5]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

conversation = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

print(
    conversation.invoke(
        input={
            "question": "Hi, I'm learning about space. Can you tell me about planets?"
        },
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "What's the largest planet in our solar system?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "How does its size compare to Earth?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)

Certainly! Planets are celestial bodies that orbit a star, such as our Sun, and have enough mass for their gravity to form a nearly round shape. Here are some key characteristics and categories of planets:

### Types of Planets
1. **Terrestrial Planets**:
   - **Mercury**: The closest planet to the Sun; small and rocky with a thin atmosphere.
   - **Venus**: Similar in size and composition to Earth; has a thick, toxic atmosphere.
   - **Earth**: The only known planet to support life; has liquid water and a breathable atmosphere.
   - **Mars**: Known as the "Red Planet"; cold, desert-like with evidence of past water.

2. **Gas Giants**:
   - **Jupiter**: The largest planet in our solar system; composed mostly of hydrogen and helium with a Great Red Spot.
   - **Saturn**: Famous for its extensive ring system; also a gas giant.
   - **Uranus**: Has a blue-green color due to methane in its atmosphere; rotates on its side.
   - **Neptune**: The farthest known planet from the Sun; has strong

Let's compare how single-turn and multi-turn prompts handle a series of related questions:

In [7]:
# Single-turn prompts
prompts = [
    "What is the capital of France?",
    "What is its population?",
    "What is the city's most famous landmark?",
]

print("Single-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {llm.invoke(prompt).content}\n")

# Multi-turn prompts
print("Multi-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    response = conversation.invoke(
        input={"question": prompt}, config={"configurable": {"session_id": "bar"}}
    ).content
    print(f"A: {response}\n")

Single-turn responses:
Q: What is the capital of France?
A: The capital of France is Paris. Paris is not only the capital but also the largest city in France, known for its significant cultural, economic, and political influence. It is situated in the northern part of the country along the Seine River. Paris is renowned for its historical landmarks, such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, which attract millions of visitors annually. Additionally, it is a global center for art, fashion, gastronomy, and diplomacy.

Q: What is its population?
A: It seems you didn't specify which location or entity you're referring to when you mention "its population." Populations can vary greatly depending on whether you're talking about a city, country, region, or even an organization.

Here are a few examples of populations for different types of entities as of the most recent data available:

- **World Population**: Approximately 8 billion people as of 2023.
- **United St